In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
# let's copy the training trec files locally
bucket = "gs://dsgt-clef-erisk-2024"
dst = "/mnt/data/task1/train"
! mkdir -p {dst}
# copy files quietly
! gcloud storage rsync -r "{bucket}/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/" {dst}/ &> /dev/null


In [2]:
from erisk.utils import get_spark

bucket = "gs://dsgt-clef-erisk-2024"
dst = "/mnt/data/task1/train"
spark = get_spark(**{"spark.jars.packages": "com.databricks:spark-xml_2.12:0.17.0"})
spark

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/anthony/.ivy2/cache
The jars for the packages stored in: /home/anthony/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-97597804-e448-4bef-af14-41e2c941b6a7;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.17.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
downloading https://repo1.maven.org/maven2/com/databricks/spark-xml_2.12/0.17.0/spark-xml_2.12-0.17.0.jar ...
	[SUCCESSFUL ] com.databricks#spark-xml_2.12;0.17.0!spark-xml_2.12.jar (53ms)
downloading https://repo1.maven.org/maven2/commons-io/commons-io/2.11.0/commons-io-2.11.0.jar ...
	[SUCCESSFUL ] commons-io#commons-io;2.11.0!commons-io.jar (71ms)
downloading https://repo1.maven.org/maven2/org/glassfish/jaxb/txw2/3

In [3]:
! ls -R {dst} | head

/mnt/data/task1/train:
g_qrels_majority_2.csv
g_rels_consenso.csv
new_data

/mnt/data/task1/train/new_data:
s_0.trec
s_1.trec
s_10.trec
s_100.trec
ls: write error: Broken pipe


In [4]:
! cat {dst}/new_data/s_0.trec | head

        <DOC>
            	<DOCNO>s_0_0_0</DOCNO>
            	<TEXT>1.ye katiliyorum</TEXT>
        </DOC>
        <DOC>
            	<DOCNO>s_0_1_0</DOCNO>
            	<TEXT>ok haklsn abi gold atar msn</TEXT>
        </DOC>
        <DOC>
            	<DOCNO>s_0_2_0</DOCNO>


In [18]:
# read trec files into spark with text
from pyspark.sql import functions as F
from pathlib import Path

# https://stackoverflow.com/questions/50429315/read-xml-in-spark
df = (
    spark.read.format("com.databricks.spark.xml")
    .option("rowTag", "DOC")
    .load(f"{dst}/new_data/*.trec")
    # get the filename
    .withColumn("filename", F.udf(lambda p: Path(p).name)(F.input_file_name()))
).cache()

df.show(truncate=80)

24/03/17 11:21:08 WARN BlockManager: Putting block rdd_107_2439 failed due to exception org.apache.spark.TaskKilledException.
24/03/17 11:21:08 WARN BlockManager: Block rdd_107_2439 could not be removed as it was not found on disk or in memory
24/03/17 11:21:08 WARN TaskSetManager: Lost task 2439.0 in stage 17.0 (TID 8664) (erisk-dev.us-central1-a.c.dsgt-clef-2024.internal executor driver): TaskKilled (Stage cancelled: Job 17 cancelled )
24/03/17 11:21:08 WARN BlockManager: Putting block rdd_107_2440 failed due to exception org.apache.spark.TaskKilledException.
24/03/17 11:21:08 WARN BlockManager: Block rdd_107_2440 could not be removed as it was not found on disk or in memory
24/03/17 11:21:08 WARN TaskSetManager: Lost task 2440.0 in stage 17.0 (TID 8665) (erisk-dev.us-central1-a.c.dsgt-clef-2024.internal executor driver): TaskKilled (Stage cancelled: Job 17 cancelled )
24/03/17 11:21:08 WARN BlockManager: Putting block rdd_107_2441 failed due to exception org.apache.spark.TaskKilledE

+--------+--------------------------------------------------------------------------------+--------+
|   DOCNO|                                                                            TEXT|filename|
+--------+--------------------------------------------------------------------------------+--------+
| s_0_0_0|                                                                1.ye katiliyorum|s_0.trec|
| s_0_1_0|                                                     ok haklsn abi gold atar msn|s_0.trec|
| s_0_2_0|                                                             almanca yarrak gibi|s_0.trec|
| s_0_3_0|hani u oyunlarn en gl en gizemli silah, zellii falan olan eylere garip isimle...|s_0.trec|
| s_0_3_1|dead cellste ygdar orus li ox var mesela, deus ex machina da olabilir onun gi...|s_0.trec|
| s_0_3_2|                bunlarn bir dili falan var m yoksa kendileri mi uydurmu yapmclar|s_0.trec|
| s_0_4_0|                                                         lnce diriltiyor sanirim|

In [22]:
# write this to gcs
df.repartition(1).write.mode("overwrite").parquet(f"{bucket}/task1/parquet/train")

In [23]:
! gcloud storage du --readable-sizes {bucket}/task1/parquet/train

0B           gs://dsgt-clef-erisk-2024/task1/parquet/train/
0B           gs://dsgt-clef-erisk-2024/task1/parquet/train/_SUCCESS
210.82MiB    gs://dsgt-clef-erisk-2024/task1/parquet/train/part-00000-2c958078-9a2d-4198-98f9-f6ec6156d701-c000.snappy.parquet
210.82MiB    gs://dsgt-clef-erisk-2024/task1/parquet/train/


## test parquet

We might as well do the test parquet while we're at it, since the process is exactly the same.
Let's see if we can do it from gcs directly this time.


In [27]:
! gcloud storage cat gs://dsgt-clef-erisk-2024/task1/dataset/s_0.trec | head -n 20

        <DOC>
            	<DOCNO>0_0_0</DOCNO>
            	<PRE></PRE>
            	<TEXT>I guess it depends on what cheating entails.</TEXT>
            	<POST>I met someone who understands me better than I understand myself.</POST>
        </DOC>
        <DOC>
            	<DOCNO>0_0_1</DOCNO>
            	<PRE></PRE>
            	<TEXT>I met someone who understands me better than I understand myself.</TEXT>
            	<POST>Our friendship thus far is completely platonic, but I know if I had the opportunity, guilt-free I would pounce on dat ass.</POST>
        </DOC>
        <DOC>
            	<DOCNO>0_0_2</DOCNO>
            	<PRE>I met someone who understands me better than I understand myself.</PRE>
            	<TEXT>Our friendship thus far is completely platonic, but I know if I had the opportunity, guilt-free I would pounce on dat ass.</TEXT>
            	<POST>

I tried to break up with my sweet SO, saying that I just hadn't been feeling the same way I had when we met (not

In [24]:
df = (
    spark.read.format("com.databricks.spark.xml")
    .option("rowTag", "DOC")
    .load(f"{bucket}/task1/dataset/*.trec")
    # get the filename
    .withColumn("filename", F.udf(lambda p: Path(p).name)(F.input_file_name()))
).cache()

df.show(truncate=80)

+------+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+---------------+--------+
| DOCNO|                                                                            POST|                                                                             PRE|                                                                            TEXT|_corrupt_record|filename|
+------+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+---------------+--------+
| 0_0_0|               I met someone who understands me better than I understand myself.|                                                                                

In [25]:
df.repartition(8).write.mode("overwrite").parquet(f"{bucket}/task1/parquet/test")